# Importing the libraries

# New Section

# Importing the dataset

In [ ]:
!git clone https://github.com/mandar196/Hate_Speech_Detection-NLP.git

Cloning into 'Hate_Speech_Detection-NLP'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!gdown --id 1OPiQvG-AAtFNb9jJQnduvDZjz-CrZjxn #training data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
dataset = pd.read_csv('/content/people.csv')
dataset.head()

,id,set_name,img,text,gold_hate,gold_pc,gold_attack,pc,attack
0,42953,train,img/42953.png,its their character not their color that matters,['not_hateful'],['pc_empty'],['attack_empty'],None,None
1,23058,train,img/23058.png,don't be afraid to love again everyone is not ...,['not_hateful'],['pc_empty'],['attack_empty'],None,None
2,13894,train,img/13894.png,putting bows on your pet,['not_hateful'],['pc_empty'],['attack_empty'],None,None
3,37408,train,img/37408.png,i love everything and everybody! except for sq...,['not_hateful'],['pc_empty'],['attack_empty'],None,None
4,82403,train,img/82403.png,"everybody loves chocolate chip cookies, even h...",['not_hateful'],['pc_empty'],['attack_empty'],None,None


In [3]:
print(dataset['gold_hate'].unique())

["['not_hateful']" "['hateful']"]


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           8500 non-null   int64 
 1   set_name     8500 non-null   object
 2   img          8500 non-null   object
 3   text         8500 non-null   object
 4   gold_hate    8500 non-null   object
 5   gold_pc      8500 non-null   object
 6   gold_attack  8500 non-null   object
 7   pc           8500 non-null   object
 8   attack       8500 non-null   object
dtypes: int64(1), object(8)
memory usage: 597.8+ KB


In [5]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
id,8500.0,50006.595765,28429.195025,1235.0,25840.0,49834.0,74332.25,98764.0


In [6]:
dataset['gold_hate'] = dataset['gold_hate'].replace({"['hateful']": 0})
dataset['gold_hate'] = dataset['gold_hate'].replace({"['not_hateful']": 1})
# dataset['Class'] = dataset['Class'].replace({'racism': 0})

In [7]:
dicti = { 'gold_hate': 'class','text': 'tweet'}
dataset.rename(columns=dicti,
          inplace=True)

In [8]:
dt_trasformed = dataset[['class', 'tweet']]
y = dt_trasformed.iloc[:, :-1].values
print(y)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


### Encoding the Dependent Variable

In [ ]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# y = np.array(ct.fit_transform(y))

In [ ]:
print(y)

[[0]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]


I separated this y in two variables that we will use to fit hate speech models and offensive speech models

In [ ]:
# y_df = pd.DataFrame(y)
# y_hate = np.array(y_df[0])
# y_offensive = np.array(y_df[1])
# y_neither = np.array(y_df[2])
# print(sum(y_neither==2))

0


In [9]:
#extra
xy = np.ravel(y,order='C')

In [ ]:
# print(y_hate)
# print(y_offensive)

[0. 1. 1. ... 1. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


## Cleaning the texts

In [11]:
corpus = []
for i in range(0, 8500):
  review = re.sub('[^a-zA-Z]', ' ', dt_trasformed['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [12]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

## Splitting the dataset into the Training set and Test set

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, xy, test_size = 0.30, random_state = 0,shuffle=True)

In [21]:
dfd = pd.DataFrame(y_test)
dfd.to_csv('ML_FB!_ytest.csv')
# print(type(data))
dfdd = pd.DataFrame(X_test)
dfdd.to_csv('ML_FB!_Xtest.csv')

## Finding the best models to predict hate speech

**Naive Bayes**

In [14]:
classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

GaussianNB()

**Decision Tree**

In [15]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

**KNN**

In [16]:
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

**Logistic Regression**

In [17]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

**Random Forest**

In [18]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

SVM Classifier

In [19]:
classifier_svm = svm.SVC()
classifier_svm.fit(X_train, y_train)

SVC()

XGBoost Classifier

In [20]:
classifier_xgb= XGBClassifier()
classifier_xgb.fit(X_train, y_train)

XGBClassifier()

# Making the Confusion Matrix for each model

In [22]:
from sklearn.metrics import classification_report

In [23]:
#Naive Bayes
y_pred_np = classifier_np.predict(X_test)
cm = confusion_matrix(y_test, y_pred_np)
print(classification_report(y_test,y_pred_np))
print(y_pred_np)
print(cm)


              precision    recall  f1-score   support

           0       0.38      0.84      0.52       929
           1       0.70      0.21      0.32      1621

    accuracy                           0.44      2550
   macro avg       0.54      0.53      0.42      2550
weighted avg       0.58      0.44      0.40      2550

[0 0 1 ... 1 0 0]
[[ 783  146]
 [1280  341]]


In [24]:
#KNN
y_pred_knn = classifier_knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred_knn)
print(classification_report(y_test,y_pred_knn))
print(cm)

              precision    recall  f1-score   support

           0       0.52      0.27      0.35       929
           1       0.67      0.86      0.75      1621

    accuracy                           0.64      2550
   macro avg       0.59      0.56      0.55      2550
weighted avg       0.62      0.64      0.61      2550

[[ 248  681]
 [ 231 1390]]


In [25]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred_xgb)
print(classification_report(y_test,y_pred_xgb))
print(cm)

              precision    recall  f1-score   support

           0       0.77      0.37      0.50       929
           1       0.72      0.94      0.82      1621

    accuracy                           0.73      2550
   macro avg       0.75      0.65      0.66      2550
weighted avg       0.74      0.73      0.70      2550

[[ 343  586]
 [ 101 1520]]


In [26]:
#SVM
y_pred_svm = classifier_svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(classification_report(y_test,y_pred_svm))
print(cm)

              precision    recall  f1-score   support

           0       0.71      0.43      0.54       929
           1       0.73      0.90      0.81      1621

    accuracy                           0.73      2550
   macro avg       0.72      0.67      0.67      2550
weighted avg       0.73      0.73      0.71      2550

[[ 402  527]
 [ 161 1460]]


In [27]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(classification_report(y_test,y_pred_lr))
print(cm)

              precision    recall  f1-score   support

           0       0.66      0.49      0.56       929
           1       0.74      0.86      0.80      1621

    accuracy                           0.72      2550
   macro avg       0.70      0.67      0.68      2550
weighted avg       0.71      0.72      0.71      2550

[[ 452  477]
 [ 235 1386]]


In [28]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_dt)
print(classification_report(y_test,y_pred_dt))
print(cm)


              precision    recall  f1-score   support

           0       0.55      0.58      0.56       929
           1       0.75      0.73      0.74      1621

    accuracy                           0.67      2550
   macro avg       0.65      0.65      0.65      2550
weighted avg       0.68      0.67      0.68      2550

[[ 539  390]
 [ 441 1180]]


In [29]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
print(classification_report(y_test,y_pred_rf))
print(cm)

              precision    recall  f1-score   support

           0       0.58      0.52      0.55       929
           1       0.74      0.78      0.76      1621

    accuracy                           0.69      2550
   macro avg       0.66      0.65      0.65      2550
weighted avg       0.68      0.69      0.68      2550

[[ 482  447]
 [ 356 1265]]


In [30]:
rf_score = accuracy_score(y_test, y_pred_rf)
knn_score = accuracy_score(y_test, y_pred_knn)
svm_score = accuracy_score(y_test, y_pred_svm)
xgb_score = accuracy_score(y_test, y_pred_xgb)
lr_score = accuracy_score(y_test, y_pred_lr)
dt_score = accuracy_score(y_test, y_pred_dt)
np_score = accuracy_score(y_test, y_pred_np)

print('Random Forest Accuracy: ', str(rf_score))
print('K Nearest Neighbours Accuracy: ', str(knn_score))
print('Support Vector Machine Accuracy: ', str(svm_score))
print('XGBoost Classifier Accuracy: ', str(xgb_score))
print('Logistic Regression Accuracy: ',str(lr_score))
print('Decision Tree Accuracy: ', str(dt_score))
print('Naive Bayes Accuracy: ', str(np_score))

Random Forest Accuracy:  0.6850980392156862
K Nearest Neighbours Accuracy:  0.6423529411764706
Support Vector Machine Accuracy:  0.7301960784313726
XGBoost Classifier Accuracy:  0.7305882352941176
Logistic Regression Accuracy:  0.7207843137254902
Decision Tree Accuracy:  0.6741176470588235
Naive Bayes Accuracy:  0.4407843137254902


So XGBoost Classifier looks better to predict hate speech based on this dataset. It's important to emphasize Random Forest,KNN and Decision Tree had great results as well. This Dataset looks like a product of artificial intelligence to classify hate and offensive speech.

In [ ]:
#Now finally saving the model with XGB Classifier.
import joblib
joblib.dump(classifier_xgb, 'hatespeech.pkl')

['hatespeech.pkl']

In [31]:
from sklearn.metrics import precision_recall_fscore_support

In [32]:
score_rf=precision_recall_fscore_support(y_test,y_pred_rf,average='weighted')
print(score_rf)

(0.6792553798781796, 0.6850980392156862, 0.6812881999275529, None)


In [33]:
score_knn=precision_recall_fscore_support(y_test,y_pred_knn,average='weighted')
print(score_knn)

(0.6152774040890165, 0.6423529411764706, 0.6069964687342483, None)


In [34]:
score_svm=precision_recall_fscore_support(y_test,y_pred_svm,average='weighted')
print(score_svm)

(0.7272186938781248, 0.7301960784313726, 0.7107881286895907, None)


In [35]:
score_xgb=precision_recall_fscore_support(y_test,y_pred_xgb,average='weighted')
print(score_xgb)

(0.7402454666833751, 0.7305882352941176, 0.7005340591050615, None)


In [36]:
score_lr=precision_recall_fscore_support(y_test,y_pred_lr,average='weighted')
print(score_lr)

(0.7126200583813468, 0.7207843137254902, 0.7095749089327789, None)


In [37]:
score_dt=precision_recall_fscore_support(y_test,y_pred_dt,average='weighted')
print(score_dt)

(0.6781494941925815, 0.6741176470588235, 0.6758665121799022, None)


In [38]:
score_np=precision_recall_fscore_support(y_test,y_pred_np,average='weighted')
print(score_np)

(0.5833841404073805, 0.4407843137254902, 0.39634345181923036, None)


# Testing

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Test on Facebook

In [40]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [41]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.38      0.41      0.39        59
           1       0.59      0.56      0.57        90

    accuracy                           0.50       149
   macro avg       0.48      0.48      0.48       149
weighted avg       0.50      0.50      0.50       149

[[24 35]
 [40 50]]
(0.5037998420844848, 0.4966442953020134, 0.4996843065266703, None)


In [42]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.56      0.15      0.24        59
           1       0.62      0.92      0.74        90

    accuracy                           0.62       149
   macro avg       0.59      0.54      0.49       149
weighted avg       0.60      0.62      0.54       149

[[ 9 50]
 [ 7 83]]
(0.5996839834485543, 0.6174496644295302, 0.5446678905709211, None)


In [43]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.29      0.03      0.06        59
           1       0.60      0.94      0.73        90

    accuracy                           0.58       149
   macro avg       0.44      0.49      0.40       149
weighted avg       0.47      0.58      0.47       149

[[ 2 57]
 [ 5 85]]
(0.4747005523071314, 0.5838926174496645, 0.46660425126058086, None)


In [44]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.50      0.03      0.06        59
           1       0.61      0.98      0.75        90

    accuracy                           0.60       149
   macro avg       0.55      0.51      0.41       149
weighted avg       0.56      0.60      0.48       149

[[ 2 57]
 [ 2 88]]
(0.5645683869474658, 0.6040268456375839, 0.4775187051354408, None)


In [45]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.38      0.46      0.41        59
           1       0.58      0.50      0.54        90

    accuracy                           0.48       149
   macro avg       0.48      0.48      0.48       149
weighted avg       0.50      0.48      0.49       149

[[27 32]
 [45 45]]
(0.5014926348818965, 0.48322147651006714, 0.4887490248254963, None)


In [46]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.33      0.53      0.41        59
           1       0.50      0.31      0.38        90

    accuracy                           0.40       149
   macro avg       0.42      0.42      0.40       149
weighted avg       0.43      0.40      0.39       149

[[31 28]
 [62 28]]
(0.4340044742729306, 0.3959731543624161, 0.3931968954287898, None)


In [47]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.37      0.58      0.45        59
           1       0.56      0.36      0.44        90

    accuracy                           0.44       149
   macro avg       0.47      0.47      0.44       149
weighted avg       0.49      0.44      0.44       149

[[34 25]
 [58 32]]
(0.48544069540644724, 0.4429530201342282, 0.44129671069281085, None)


## Test on stormfront

In [48]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [49]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.09      0.48      0.15       342
           1       0.87      0.41      0.56      2942

    accuracy                           0.42      3284
   macro avg       0.48      0.45      0.35      3284
weighted avg       0.79      0.42      0.52      3284

[[ 164  178]
 [1728 1214]]
(0.7903291100205473, 0.41961023142509135, 0.5171695300312762, None)


In [50]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.10      0.16      0.12       342
           1       0.90      0.84      0.86      2942

    accuracy                           0.77      3284
   macro avg       0.50      0.50      0.49      3284
weighted avg       0.81      0.77      0.79      3284

[[  55  287]
 [ 483 2459]]
(0.8128738599534892, 0.7655298416565165, 0.7876015450609956, None)


In [51]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.24      0.08      0.12       342
           1       0.90      0.97      0.93      2942

    accuracy                           0.88      3284
   macro avg       0.57      0.52      0.53      3284
weighted avg       0.83      0.88      0.85      3284

[[  26  316]
 [  83 2859]]
(0.8315371202531117, 0.8785018270401949, 0.8494310111681533, None)


In [52]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.14      0.05      0.07       342
           1       0.90      0.96      0.93      2942

    accuracy                           0.87      3284
   macro avg       0.52      0.51      0.50      3284
weighted avg       0.82      0.87      0.84      3284

[[  17  325]
 [ 106 2836]]
(0.8181440016294367, 0.8687576126674786, 0.84020689761777, None)


In [53]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.14      0.24      0.18       342
           1       0.90      0.83      0.87      2942

    accuracy                           0.77      3284
   macro avg       0.52      0.53      0.52      3284
weighted avg       0.82      0.77      0.79      3284

[[  81  261]
 [ 492 2450]]
(0.8243319594034851, 0.7707064555420219, 0.7949652262079461, None)


In [54]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.13      0.54      0.21       342
           1       0.92      0.59      0.72      2942

    accuracy                           0.58      3284
   macro avg       0.52      0.56      0.47      3284
weighted avg       0.83      0.58      0.67      3284

[[ 184  158]
 [1206 1736]]
(0.8349105931984725, 0.584652862362972, 0.6653076294841392, None)


In [55]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.13      0.46      0.21       342
           1       0.91      0.65      0.76      2942

    accuracy                           0.63      3284
   macro avg       0.52      0.55      0.48      3284
weighted avg       0.83      0.63      0.70      3284

[[ 158  184]
 [1037 1905]]
(0.8307204052107241, 0.6281973203410475, 0.6998489820558075, None)


## Test on Reddit

In [56]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [57]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

         0.0       0.37      0.28      0.32      5813
         1.0       0.45      0.56      0.50      6188

    accuracy                           0.42     12001
   macro avg       0.41      0.42      0.41     12001
weighted avg       0.41      0.42      0.41     12001

[[1615 4198]
 [2743 3445]]
(0.4119133767663283, 0.4216315307057745, 0.4106843709467332, None)


In [58]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

         0.0       0.46      0.13      0.21      5813
         1.0       0.51      0.85      0.64      6188

    accuracy                           0.50     12001
   macro avg       0.49      0.49      0.42     12001
weighted avg       0.49      0.50      0.43     12001

[[ 784 5029]
 [ 924 5264]]
(0.486034627488406, 0.5039580034997083, 0.4303626366943586, None)


In [59]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

         0.0       0.57      0.08      0.15      5813
         1.0       0.52      0.94      0.67      6188

    accuracy                           0.53     12001
   macro avg       0.54      0.51      0.41     12001
weighted avg       0.54      0.53      0.42     12001

[[ 486 5327]
 [ 370 5818]]
(0.5441779531339381, 0.5252895592033997, 0.41674616728176295, None)


In [60]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

         0.0       0.42      0.03      0.06      5813
         1.0       0.51      0.96      0.67      6188

    accuracy                           0.51     12001
   macro avg       0.47      0.50      0.36     12001
weighted avg       0.47      0.51      0.37     12001

[[ 186 5627]
 [ 252 5936]]
(0.47039544459788984, 0.5101241563203066, 0.3736784057015806, None)


In [61]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

         0.0       0.53      0.35      0.42      5813
         1.0       0.54      0.71      0.61      6188

    accuracy                           0.53     12001
   macro avg       0.53      0.53      0.52     12001
weighted avg       0.53      0.53      0.52     12001

[[2048 3765]
 [1818 4370]]
(0.5335819723463758, 0.5347887676026998, 0.5196177791691238, None)


In [62]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

         0.0       0.53      0.54      0.53      5813
         1.0       0.56      0.55      0.56      6188

    accuracy                           0.55     12001
   macro avg       0.55      0.55      0.55     12001
weighted avg       0.55      0.55      0.55     12001

[[3121 2692]
 [2759 3429]]
(0.5459521004899268, 0.5457878510124157, 0.5458529741841505, None)


In [63]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

         0.0       0.56      0.56      0.56      5813
         1.0       0.59      0.60      0.59      6188

    accuracy                           0.58     12001
   macro avg       0.58      0.58      0.58     12001
weighted avg       0.58      0.58      0.58     12001

[[3242 2571]
 [2504 3684]]
(0.5769803961376134, 0.5771185734522123, 0.5770315030966351, None)


## Test on Davidson

In [64]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [65]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.05      0.52      0.10       427
           1       0.94      0.42      0.58      7008

    accuracy                           0.43      7435
   macro avg       0.49      0.47      0.34      7435
weighted avg       0.88      0.43      0.55      7435

[[ 223  204]
 [4042 2966]]
(0.8849143416820877, 0.4289172831203766, 0.5548125332401604, None)


In [66]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.06      0.19      0.09       427
           1       0.94      0.83      0.88      7008

    accuracy                           0.79      7435
   macro avg       0.50      0.51      0.49      7435
weighted avg       0.89      0.79      0.84      7435

[[  79  348]
 [1219 5789]]
(0.892615761658952, 0.7892400806993948, 0.8354667493615885, None)


In [67]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.05      0.02      0.03       427
           1       0.94      0.97      0.96      7008

    accuracy                           0.92      7435
   macro avg       0.49      0.50      0.49      7435
weighted avg       0.89      0.92      0.90      7435

[[  10  417]
 [ 204 6804]]
(0.8908209340870838, 0.9164761264290517, 0.9032240673177472, None)


In [68]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.06      0.03      0.04       427
           1       0.94      0.97      0.96      7008

    accuracy                           0.92      7435
   macro avg       0.50      0.50      0.50      7435
weighted avg       0.89      0.92      0.90      7435

[[  12  415]
 [ 195 6813]]
(0.8917801017489297, 0.917955615332885, 0.9043547330862248, None)


In [69]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.07      0.26      0.10       427
           1       0.94      0.77      0.85      7008

    accuracy                           0.74      7435
   macro avg       0.51      0.52      0.48      7435
weighted avg       0.89      0.74      0.81      7435

[[ 111  316]
 [1584 5424]]
(0.894439349035831, 0.7444519166106254, 0.8080939816672921, None)


In [70]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.06      0.36      0.10       427
           1       0.94      0.66      0.78      7008

    accuracy                           0.65      7435
   macro avg       0.50      0.51      0.44      7435
weighted avg       0.89      0.65      0.74      7435

[[ 154  273]
 [2354 4654]]
(0.893868625372432, 0.6466711499663752, 0.7411279361539053, None)


In [71]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.07      0.38      0.12       427
           1       0.95      0.69      0.80      7008

    accuracy                           0.67      7435
   macro avg       0.51      0.54      0.46      7435
weighted avg       0.90      0.67      0.76      7435

[[ 164  263]
 [2188 4820]]
(0.8978039268733383, 0.6703429724277068, 0.758276657953853, None)


## Test on Waseem

In [72]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [73]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.25      0.50      0.33       898
           1       0.72      0.46      0.56      2500

    accuracy                           0.47      3398
   macro avg       0.49      0.48      0.45      3398
weighted avg       0.60      0.47      0.50      3398

[[ 452  446]
 [1349 1151]]
(0.5965828356726145, 0.4717480871100647, 0.5019015473101053, None)


In [74]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.22      0.17      0.19       898
           1       0.73      0.79      0.76      2500

    accuracy                           0.63      3398
   macro avg       0.48      0.48      0.47      3398
weighted avg       0.59      0.63      0.61      3398

[[ 152  746]
 [ 524 1976]]
(0.5935135926085815, 0.6262507357268982, 0.6078380746410441, None)


In [75]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.23      0.03      0.05       898
           1       0.73      0.96      0.83      2500

    accuracy                           0.72      3398
   macro avg       0.48      0.50      0.44      3398
weighted avg       0.60      0.72      0.63      3398

[[  28  870]
 [  95 2405]]
(0.6004416206071401, 0.7160094173042967, 0.6272821527724596, None)


In [76]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.20      0.13      0.16       898
           1       0.72      0.81      0.77      2500

    accuracy                           0.63      3398
   macro avg       0.46      0.47      0.46      3398
weighted avg       0.58      0.63      0.60      3398

[[ 117  781]
 [ 466 2034]]
(0.5846410946623168, 0.6330194231901118, 0.6048668939489765, None)


In [77]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.29      0.41      0.34       898
           1       0.75      0.64      0.69      2500

    accuracy                           0.58      3398
   macro avg       0.52      0.53      0.52      3398
weighted avg       0.63      0.58      0.60      3398

[[ 371  527]
 [ 888 1612]]
(0.6323364034413661, 0.5835785756327251, 0.6022225085277519, None)


In [78]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.23      0.57      0.33       898
           1       0.67      0.31      0.42      2500

    accuracy                           0.38      3398
   macro avg       0.45      0.44      0.38      3398
weighted avg       0.55      0.38      0.40      3398

[[ 514  384]
 [1723  777]]
(0.5531082652850435, 0.3799293702177752, 0.39895505129620723, None)


In [79]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.24      0.49      0.32       898
           1       0.70      0.43      0.54      2500

    accuracy                           0.45      3398
   macro avg       0.47      0.46      0.43      3398
weighted avg       0.58      0.45      0.48      3398

[[ 436  462]
 [1413 1087]]
(0.5786076712686352, 0.44820482636845205, 0.4789185849904639, None)
